In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage,BaseMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
TAVILY_API_KEY=""

In [4]:
tool = TavilySearchResults(tavily_api_key=TAVILY_API_KEY,max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


# let us now call the agent state 

In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [6]:
class Agent:
    def __init__(self,model,tools,system=""):
        self.system=system
        graph =StateGraph(AgentState)
        graph.add_node("llm",self.call_openai)
        graph.add_node("action",self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True:"action",False:END}
        )
        graph.add_edge("action","llm")
        graph.set_entry_point("llm")
        self.graph=graph.compile()## compile the graph 
        self.tools={t.name:t for  t in tools}## pass the tools
        self.model=model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls## take the last message from the list of messages 
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}    

In [7]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [16]:
messages = [HumanMessage(content="What is the weather in New Delhi?")]

In [17]:
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in New Delhi'}, 'id': 'call_rV1TuDVB8vA5kCcH73OAjCGC'}
Back to the model!


In [18]:
result

{'messages': [HumanMessage(content='What is the weather in New Delhi?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rV1TuDVB8vA5kCcH73OAjCGC', 'function': {'arguments': '{"query":"weather in New Delhi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 154, 'total_tokens': 175}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3e1d45a0-6123-4dd7-a68e-716a6f201d02-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in New Delhi'}, 'id': 'call_rV1TuDVB8vA5kCcH73OAjCGC'}]),
  ToolMessage(content='[{\'url\': \'https://www.timeanddate.com/weather/india/new-delhi/historic?month=6&year=2024\', \'content\': \'June 2024 Weather in New Delhi — Graph °F. See Hour-by-hour Forecast for upcoming weather. See weather overview. High & Low Weather Summary for June 2024 Temperature Humidity

In [14]:
result['messages'][-1].content

'The current weather in San Francisco is sunny with a temperature of 60.1°F. The wind speed is 6.1 kph coming from the west. The humidity is at 67%, and there is no precipitation at the moment.'

In [19]:
messages = [HumanMessage(content="What is the weather in San francisco and New delhi?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_JfuVfBH4LPITfiV2CnB9cKx8'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in New Delhi'}, 'id': 'call_LNLjVIL6SWr9ysK4vbt4JE86'}
Back to the model!


In [20]:
result['messages'][-1].content

'The current weather in San Francisco is sunny with a temperature of 60.1°F. The wind speed is 6.1 km/h coming from the west. The humidity is at 67% with clear skies.\n\nIn New Delhi, the weather is misty with a temperature of 97.2°F. The wind speed is 6.8 km/h from the east. The humidity is at 57% with partly cloudy skies and some precipitation.'

# testing another pattern

In [21]:
query = "Who won world cup T20 worls cup series in 2007 and 2024? " 
messages = [HumanMessage(content=query)]

In [22]:
model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'winner of 2007 T20 World Cup'}, 'id': 'call_RhP2SadKTeYLvO7Hdzjs0qF8'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'winner of 2024 T20 World Cup'}, 'id': 'call_NbLqz5C2ojr8GcSv9yHw0XS6'}
Back to the model!


In [23]:
print(result['messages'][-1].content)

### Winners of the T20 World Cup

- **2007**: India won the inaugural T20 World Cup, defeating Pakistan in the final.
  - Source: [Wikipedia](https://en.wikipedia.org/wiki/2007_ICC_World_Twenty20), [ESPN Cricinfo](https://www.espncricinfo.com/series/icc-world-twenty20-2007-08-286109/india-vs-pakistan-final-287879/full-scorecard)

- **2024**: India also won the 2024 T20 World Cup, defeating South Africa in a dramatic final.
  - Source: [CNN](https://www.cnn.com/2024/06/29/sport/india-south-africa-t20-mens-world-cup-final-spt-intl/index.html), [Al Jazeera](https://www.aljazeera.com/sports/2024/6/29/india-vs-south-africa-icc-t20-world-cup-final-result-match-report-champions-rohit-markram-bumrah-shamsi)
